# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Kraus  ->  S. Kraus  |  ['S. Kraus']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Liu  ->  J. Liu  |  ['J. Liu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']


Arxiv has 102 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.09259


extracting tarball to tmp_2410.09259...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.09499


extracting tarball to tmp_2410.09499...

...

extracting tarball to tmp_2410.10183...

 done.


L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure GJ1214b_retrieval_steam.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.75\linewidth]{GJ1214b_retrieval_steam.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 120 bibliographic references in tmp_2410.10183/ms.bbl.
Issues with the citations
syntax error in line 5: unbalanced braces


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.09705-b31b1b.svg)](https://arxiv.org/abs/2410.09705) | **The formation and survival of the Milky Way's oldest stellar disk**  |
|| M. Xiang, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2024-10-15*|
|*Comments*| *Published in Nature Astronomy on 10 October, 2024; This manuscript is the accepted version*|
|**Abstract**|            It remains a mystery when our Milky Way first formed a stellar disk component that survived and maintained its disk structure from subsequent galaxy mergers. We present a study of the age-dependent structure and star formation rate of the Milky Way's disk using high-alpha stars with significant orbital angular momentum that have precise age determinations. Our results show that the radial scale length is nearly independent of age, while the vertical scale height experienced dramatic evolution. A disk-like geometry presents even for populations older than 13 Gyr, with the scale height-to-length ratio dropping below 0.5 for populations younger than 12.5 Gyr. We dub the oldest population that has maintained a disk geometry - apparently formed over 13 Gyr ago - PanGu. With an estimated present-day stellar mass of $2 \times 10^9$ $M_\odot$, PanGu is presumed to be a major stellar component of our Galaxy in the earliest epoch. The total present-day stellar mass of the whole high-alpha disk is $2 \times 10^{10}$ $M_\odot$, mostly formed during a distinct star formation rate peak of 11 $M_\odot$ per year around 11 Gyrs ago. A comparison with Milky Way analogs in the TNG50 simulations implies that our Galaxy has experienced an exceptionally quiescent dynamical history, even before the Gaia-Enceladus merger.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.09843-b31b1b.svg)](https://arxiv.org/abs/2410.09843) | **ALMA-IMF XVIII: The assembly of a star cluster: Dense N$_2$H$^+$ (1-0) kinematics in the massive G351.77 protocluster**  |
|| N. A. Sandoval-Garrido, et al. -- incl., <mark>P. Garcia</mark> |
|*Appeared on*| *2024-10-15*|
|*Comments*| *Submitted in A&A, 28 pages, 31 figures, 4 interactive figures, 7 tables*|
|**Abstract**|            ALMA-IMF observed 15 massive protoclusters capturing multiple spectral lines and the continuum emission. We focus on the G351.77 protocluster ($\sim$ 2500 M$_{\odot}$, estimated from single-dish continuum observations) located at 2 kpc. We trace the dense gas emission and kinematics with N$_2$H$^+$ (1-0) at $\sim$ 4 kau resolution. We estimate an N$_2$H$^+$ relative abundance $\sim (1.7 \pm 0.5) \times 10^{-10}$. We decompose the N$_2$H$^+$ emission into up to two velocity components, highlighting the kinematic complexity in the dense gas. By examining the position-velocity (PV) diagrams on small scales, we observe clear inflow signatures (V-shapes) associated with dense cores. The most prominent V-shape has a mass inflow rate of $\sim 9.8 \times 10^{-4}$ M$_{\odot}$ yr$^{-1}$ and a short timescale of $\sim$ 15.6 kyr. We also observe V-shapes without associated cores. This suggests both that cores or centers of accretion exist below the 1.3 mm detection limit, and that the V-shapes may be viable tracers of very early accretion and star formation on $\sim$ 4 kau scales. The large-scale PV diagram shows that the protocluster is separated into 2 principal velocity structures. Combined with smaller scale DCN and H$_2$CO emission, we propose a scenario of larger scale slow contraction with rotation in the center based on simple toy models. This scenario leads the suggestion of outside-in evolution of the protocluster as it collapses. The gas depletion times implied by the V-shapes are short ($\sim$ 0.3 Myr), requiring either very fast cluster formation, and/or continuous mass feeding of the protocluster. The latter is possible via the Mother Filament G351.77 is forming out of. The similarities in the properties of G351.77 and the recently published work in G353.41 indicate that many of the physical conditions inferred via the ALMA-IMF N$_2$H$^+$ observations may be generic to protoclusters.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.10183-b31b1b.svg)](https://arxiv.org/abs/2410.10183) | **Possible Carbon Dioxide Above the Thick Aerosols of GJ 1214 b**  |
|| E. Schlawin, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-10-15*|
|*Comments*| *22 pages, 11 figures, Accepted in ApJL, Please also see a companion paper Ohno et al. (2024)*|
|**Abstract**|            Sub-Neptune planets with radii smaller than Neptune (3.9 Re) are the most common type of planet known to exist in The Milky Way, even though they are absent in the Solar System. These planets can potentially have a large diversity of compositions as a result of different mixtures of rocky material, icy material and gas accreted from a protoplanetary disk. However, the bulk density of a sub-Neptune, informed by its mass and radius alone, cannot uniquely constrain its composition; atmospheric spectroscopy is necessary. GJ 1214 b, which hosts an atmosphere that is potentially the most favorable for spectroscopic detection of any sub-Neptune, is instead enshrouded in aerosols (thus showing no spectroscopic features), hiding its composition from view at previously observed wavelengths in its terminator. Here, we present a JWST NIRSpec transmission spectrum from 2.8 to 5.1 um that shows signatures of carbon dioxide and methane, expected at high metallicity. A model containing both these molecules is preferred by 3.3 and 3.6 sigma as compared to a featureless spectrum for two different data analysis pipelines, respectively. Given the low signal-to-noise of the features compared to the continuum, however, more observations are needed to confirm the carbon dioxide and methane signatures and better constrain other diagnostic features in the near-infrared. Further modeling of the planet's atmosphere, interior structure and origins will provide valuable insights about how sub-Neptunes like GJ 1214 b form and evolve.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.09259-b31b1b.svg)](https://arxiv.org/abs/2410.09259) | **Visual Orbits of Wolf-Rayet Stars I: The Orbit of the dust-producing Wolf-Rayet binary WR\,137 measured with the CHARA Array**  |
|| N. D. Richardson, et al. -- incl., <mark>S. Kraus</mark> |
|*Appeared on*| *2024-10-15*|
|*Comments*| *Accepted to ApJ*|
|**Abstract**|            Classical Wolf-Rayet stars are the descendants of massive OB stars that have lost their hydrogen envelopes and are burning helium in their cores prior to exploding as type Ib/c supernovae. The mechanisms for losing their hydrogen envelopes are either through binary interactions or through strong stellar winds potentially coupled with episodic mass-loss. Amongst the bright classical WR stars, the binary system WR\,137 (HD\,192641; WC7d + O9e) is the subject of this paper. This binary is known to have a 13-year period and produces dust near periastron. Here we report on interferometry with the CHARA Array collected over a decade of time and providing the first visual orbit for the system. We combine these astrometric measurements with archival radial velocities to measure masses of the stars of $M_{\rm WR} = 9.5\pm3.4 M_\odot$ and $M_{\rm O} = 17.3\pm 1.9 M_\odot$ when we use the most recent \textit{Gaia} distance. These results are then compared to predicted dust distribution using these orbital elements, which match the observed imaging from \textit{JWST} as discussed recently by Lau et al. Furthermore, we compare the system to the BPASS models, finding that the WR star likely formed through stellar winds and not through binary interactions. However, the companion O star did likely accrete some material from the WR's mass-loss to provide the rotation seen today that drives its status as an Oe star.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.09499-b31b1b.svg)](https://arxiv.org/abs/2410.09499) | **The SRG/eROSITA All-Sky Survey : Constraints on the structure growth from cluster number counts**  |
|| E. Artis, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-10-15*|
|*Comments*| **|
|**Abstract**|            Beyond testing the current cosmological paradigm, cluster number counts can also be utilized to investigate the discrepancies currently affecting current cosmological measurements. In particular, cosmological studies based on cosmic shear and other large-scale structure probes routinely find a value of the amplitude of the fluctuations in the universe S8 smaller than the one inferred from the primary cosmic microwave background. In this work, we investigate this tension by measuring structure evolution across cosmic time as probed by the number counts of the massive halos with the first SRG/eROSITA All-Sky Survey cluster catalog in the Western Galactic Hemisphere complemented with the overlapping Dark Energy Survey Year-3, KiloDegree Survey, and Hyper Suprime-Cam data for weak lensing mass calibration, by implementing two different parameterizations and a model-agnostic method. In the first model, we measure the cosmic linear growth index as {\gamma} = 1.19 \pm 0.21, in tension with the standard value of {\gamma} = 0.55, but in good statistical agreement with other large-scale structures probes. The second model is a phenomenological scenario in which we rescale the linear matter power spectrum at low redshift to investigate a potential reduction of structure formation, providing similar results. Finally, in a third strategy, we consider a standard {\Lambda}CDM cosmology, but we separate the cluster catalog into five redshift bins, measuring the cosmological parameters in each and inferring the evolution of the structure formation, finding hints of a reduction. Interestingly, the S8 value inferred from eRASS1 cluster number counts, when we add a degree of freedom to the matter power spectrum, recovers the value inferred by cosmic shear studies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.09720-b31b1b.svg)](https://arxiv.org/abs/2410.09720) | **Recurring tidal disruption events a decade apart in IRAS F01004-2237**  |
|| L. Sun, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-10-15*|
|*Comments*| *22 pages, 16 figures, 9 tables, submitted to A&A*|
|**Abstract**|            We report the discovery of a second optical flare that occurred in September 2021 in IRAS F01004-2237, where the first flare occurred in 2010 has been reported, and present a detailed analysis of multi-band data. The position of the flare coincides with the galaxy centre with a precision of 650 pc. The flare peaks in $\sim50$ days with an absolute magnitude of $\sim-21$ and fades in two years roughly following $L\propto t^{-5/3}$. It maintains a nearly constant blackbody temperature of $\sim$22,000 K in the late time. Its optical and UV spectra show hydrogen and helium broad emission lines with full width at half maxima of 7,000--21,000 km s$^{-1}$ and He II/H$\alpha$ ratio of 0.3--2.3. It shows weak X-ray emission relative to UV emission, with X-ray flares lasting for $<2-3$ weeks, during which the spectrum is soft with a power-law index $\Gamma=4.4^{+1.4}_{-1.3}$. These characters are consistent with a tidal disruption event (TDE), ruling out the possibilities of a supernova or an active galactic nuclei flare. With a TDE model, we infer a peak UV luminosity of $3.3\pm0.2\times10^{44}$ erg s$^{-1}$ and an energy budget of $4.5\pm0.2\times10^{51}$ erg. The two optical flares separated by $10.3\pm0.3$ years can be interpreted as repeating partial TDEs, double TDEs, or two independent TDEs. Although no definitive conclusion can be drawn, the partial TDEs interpretation predicts a third flare around 2033, and the independent TDEs interpretation predicts a high TDE rate of $\gtrsim10^{-2}$ yr$^{-1}$ in F01004-2237, both of which can be tested by future observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.09827-b31b1b.svg)](https://arxiv.org/abs/2410.09827) | **Interpreting Millimeter Emission from IMEGIN galaxies NGC 2146 and NGC 2976**  |
|| G. Ejlali, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2024-10-15*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            The millimeter continuum emission from galaxies provides important information about cold dust, its distribution, heating, and role in their InterStellar Medium (ISM). This emission also carries an unknown portion of the free-free and synchrotron radiation. The IRAM 30m Guaranteed Time Large Project, Interpreting Millimeter Emission of Galaxies with IRAM and NIKA2 (IMEGIN) provides a unique opportunity to study the origin of the millimeter emission on angular resolutions of <18" in a sample of nearby galaxies. As a pilot study, we present millimeter observations of two IMEGIN galaxies, NGC 2146 (starburst) and NGC 2976 (peculiar dwarf) at 1.15 mm and 2 mm. Combined with the data taken with Spitzer, Herschel, Plank, WSRT, and the 100m Effelsberg telescopes, we model the infrared-to-radio Spectral Energy Distribution (SED) of these galaxies, both globally and at resolved scales, using a Bayesian approach to 1) dissect different components of the millimeter emission, 2) investigate the physical properties of dust, and 3) explore correlations between millimeter emission, gas, and Star Formation Rate (SFR). We find that cold dust is responsible for most of the 1.15 mm emission in both galaxies and at 2 mm in NGC 2976. The free-free emission emits more importantly in NGC 2146 at 2 mm. The cold dust emissivity index is flatter in the dwarf galaxy ($\beta = 1.3\pm 0.1$) compared to the starburst galaxy ($\beta = 1.7\pm 0.1$). Mapping the dust-to-gas ratio, we find that it changes between 0.004 and 0.01 with a mean of $0.006\pm0.001$ in the dwarf galaxy. In addition, no global balance holds between the formation and dissociation of H$_2$ in this galaxy. We find tight correlations between the millimeter emission and both the SFR and molecular gas mass in both galaxies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.09705/./fig1.png', 'tmp_2410.09705/./fig2.png', 'tmp_2410.09705/./fig3.png']
copying  tmp_2410.09705/./fig1.png to _build/html/
copying  tmp_2410.09705/./fig2.png to _build/html/
copying  tmp_2410.09705/./fig3.png to _build/html/
exported in  _build/html/2410.09705.md
    + _build/html/tmp_2410.09705/./fig1.png
    + _build/html/tmp_2410.09705/./fig2.png
    + _build/html/tmp_2410.09705/./fig3.png
found figures ['tmp_2410.09843/./Figure/11.png', 'tmp_2410.09843/./Figure/7.1.png', 'tmp_2410.09843/./Figure/7.2.png', 'tmp_2410.09843/./Figure/7.3.png', 'tmp_2410.09843/./Figure/15.png']
copying  tmp_2410.09843/./Figure/11.png to _build/html/
copying  tmp_2410.09843/./Figure/7.1.png to _build/html/
copying  tmp_2410.09843/./Figure/7.2.png to _build/html/
copying  tmp_2410.09843/./Figure/7.3.png to _build/html/
copying  tmp_2410.09843/./Figure/15.png to _build/html/
exported in  _build/html/2410.09843.md
    + _build/html/tmp_2410.09843/./Figure/11.png
    + _build/ht

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{\ensuremath{T_{\mathrm{eff}} }}$
$\newcommand{\logg}{\ensuremath{{\mathrm{log}  } g }}$
$\newcommand{\feh}{\ensuremath{[{\mathrm{Fe/H}}] }}$
$\newcommand{\afe}{\ensuremath{[{\mathrm{\alpha/Fe}}] }}$
$\newcommand{\mk}{\ensuremath{\mathrm M_K }}$
$\newcommand{\msun}{\ensuremath{M_\odot }}$
$\newcommand{\Hii}{\ensuremath{\mathrm{H \textsc{ii}} }}$
$\newcommand{\Ha}{\ensuremath{\mathrm H_\alpha }}$
$\newcommand{\age}{\ensuremath{\tau}}$
$\newcommand{\actaa}{Acta Astron.}$
$\newcommand{\araa}{Annu. Rev. Astron. Astrophys.}$
$\newcommand{\areps}{Annu. Rev. Earth Planet. Sci.}$
$\newcommand{\aar}{Astron. Astrophys. Rev.}$
$\newcommand{\ab}{Astrobiology}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\ac}{Astron. Comput.}$
$\newcommand{\apart}{Astropart. Phys.}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\ao}{Appl. Opt.}$
$\newcommand{\apss}{Astrophys. Space Sci.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Rev.}$
$\newcommand{\aaps}{Astron. Astrophys. Suppl.}$
$\newcommand{\baas}{Bull. Am. Astron. Soc.}$
$\newcommand{\caa}{Chin. Astron. Astrophys.}$
$\newcommand{\cjaa}{Chin. J. Astron. Astrophys.}$
$\newcommand{\cqg}{Class. Quantum Gravity}$
$\newcommand{\epsl}{Earth Planet. Sci. Lett.}$
$\newcommand{\expa}{Exp. Astron.}$
$\newcommand{\frass}{Front. Astron. Space Sci.}$
$\newcommand{\gal}{Galaxies}$
$\newcommand{\gca}{Geochim. Cosmochim. Acta}$
$\newcommand{\grl}{Geophys. Res. Lett.}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\ija}{Int. J. Astrobiol.}$
$\newcommand{\jatis}{J. Astron. Telesc. Instrum. Syst.}$
$\newcommand{\jcap}{J. Cosmol. Astropart. Phys.}$
$\newcommand{\jgr}{J. Geophys. Res.}$
$\newcommand{\jgrp}{J. Geophys. Res.: Planets}$
$\newcommand{\jqsrt}{J. Quant. Spectrosc. Radiat. Transf.}$
$\newcommand{\lrca}{Living Rev. Comput. Astrophys.}$
$\newcommand{\lrr}{Living Rev. Relativ.}$
$\newcommand{\lrsp}{Living Rev. Sol. Phys.}$
$\newcommand{\memsai}{Mem. Soc. Astron. Italiana}$
$\newcommand{\maps}{Meteorit. Planet. Sci.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nat}{Nature}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\ncomms}{Nat. Commun.}$
$\newcommand{\ngeo}{Nat. Geosci.}$
$\newcommand{\nphys}{Nat. Phys.}$
$\newcommand{\na}{New Astron.}$
$\newcommand{\nar}{New Astron. Rev.}$
$\newcommand{\physrep}{Phys. Rep.}$
$\newcommand{\pra}{Phys. Rev. A}$
$\newcommand{\prb}{Phys. Rev. B}$
$\newcommand{\prc}{Phys. Rev. C}$
$\newcommand{\prd}{Phys. Rev. D}$
$\newcommand{\pre}{Phys. Rev. E}$
$\newcommand{\prl}{Phys. Rev. Lett.}$
$\newcommand{\psj}{Planet. Sci. J.}$
$\newcommand{\planss}{Planet. Space Sci.}$
$\newcommand{\pnas}{Proc. Natl Acad. Sci. USA}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\pasa}{Publ. Astron. Soc. Aust.}$
$\newcommand{\pasj}{Publ. Astron. Soc. Jpn}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\raa}{Res. Astron. Astrophys.}$
$\newcommand{\rmxaa}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{\sci}{Science}$
$\newcommand{\sciadv}{Sci. Adv.}$
$\newcommand{\solphys}{Sol. Phys.}$
$\newcommand{\sovast}{Soviet Astron.}$
$\newcommand{\ssr}{Space Sci. Rev.}$
$\newcommand{\uni}{Universe}$</div>



<div id="title">

# The formation and survival of the Milky Way's oldest stellar disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.09705-b31b1b.svg)](https://arxiv.org/abs/2410.09705)<mark>Appeared on: 2024-10-15</mark> -  _Published in Nature Astronomy on 10 October, 2024; This manuscript is the accepted version_

</div>
<div id="authors">

M. Xiang, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>J. Liu</mark>

</div>
<div id="abstract">

**Abstract:** It remains a mystery when our Milky Way first formed a stellar disk component that survived and maintained its disk structure from subsequent galaxy mergers. We present a study of the age-dependent structure and star formation rate of the Milky Way's disk using high- $\alpha$ stars with significant orbital angular momentum that have precise age determinations. Our results show that the radial scale length is nearly independent of age, while the vertical scale height experienced dramatic evolution. A disk-like geometry presents even for populations older than $13$ Gyr, with the scale height-to-length ratio dropping below 0.5 for populations younger than 12.5 Gyr. We dub the oldest population that has maintained a disk geometry -- apparently formed over 13 Gyr ago -- $* **PanGu***$ . With an estimated present-day stellar mass of $\simeq2\times10^9$  $M_\odot$ , $* **PanGu***$ is presumed to be a major stellar component of our Galaxy in the earliest epoch. The total present-day stellar mass of the whole high- $\alpha$ disk is $2\times 10^{10}M_\odot$ , mostly formed during a distinct star formation rate peak of 11 $M_\odot$ /year around 11 Gyrs ago. A comparison with Milky Way analogs in the TNG50 simulations implies that our Galaxy has experienced an exceptionally quiescent dynamical history, even before the Gaia-Enceladus merger.

</div>

<div id="div_fig1">

<img src="tmp_2410.09705/./fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Structural parameters for mono-age and mono-$\feh$ populations of the high-$\alpha$ disk.** The upper panels show the distribution of best-fit parameters in the age-[Fe/H] plane. The dashed lines delineate parameter window in age-[Fe/H] where the high-$\alpha$ disk is expected to dominate. We only retain these subsamples for the subsequent analysis, as contamination by the low-$\alpha$ stellar populations (due to measurement errors) may dominate beyond. The lower panels show the best-fit parameters as a function of age for stellar populations in the selected age windows of the upper panels. The error bars shown in the figure represent the typical range of parameter estimates (i.e., 84th - 16th percentile of the integrated likelihood function) for several populations of different metallicity. (*fig:fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2410.09705/./fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** **Scale height versus scale length of the high-$\alpha$ disk.** The left and right panel are color-coded by metalicity and age of the stellar sub-populations, respectively. The open circles without error bars in the figure refer to results where the MCMC fit failed to yield a valid parameter constraint. The scale height is smaller than the scale length for nearly all components with $\tau\lesssim 13$ Gyr, and also for some sub-populations with even older ages. The shaded regions mark the parameter space where the scale height is smaller than half the scale length. Some of the most metal-poor ($\feh\lesssim-1.5)$ sub-populations exhibit smaller scale height than scale length, but many of them have ages younger than 13 Gyr.  (*fig:fig2*)

</div>
<div id="div_fig3">

<img src="tmp_2410.09705/./fig3.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Scale height-to-length ratio as a function of age, comparing Milky Way observations to TNG 50 simulations.** The dots are our measurements, color-coded by [Fe/H] of the stellar populations. Only populations with relative uncertainties in $H_z$/$H_R$$<50$\% are shown. The solid red line is the mean observed  $H_z$/$H_R(\tau)$-relation marginalized over abundances. The solid and dashed lines in black show the average height-to-length ratio for Milky Way analogs in the TNG50 simulation, with the shaded regions indicating the variance among individual galaxies. The solid black line shows the stellar distribution morphology at present, after adding an age error of 8\% to the simulation output to mimic the observation. The dashed line shows the morphology *at birth* of the corresponding population. Populations with formal ages older than the universe result from the age uncertainties. (*fig:fig3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.09705"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\nhp}{N_2H^+\xspace}$
$\newcommand{\nthp}{N_2H^+\xspace}$
$\newcommand{\hco}{H_2CO\xspace}$
$\newcommand{\hdos}{H_2\xspace}$
$\newcommand{\tex}{T_{ex}\xspace}$
$\newcommand{\calL}{{\cal L}}$
$\newcommand{\spitzer}{\textit{Spitzer}\xspace}$
$\newcommand{\msun}{\hbox{\hbox{\rm M}_{\odot}}\xspace}$
$\newcommand{\herschel}{\textit{Herschel}\xspace}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\rsun}{R_{\odot}}$
$\newcommand{\rstar}{R_{*}}$
$\newcommand{\mdotenv}{\dot{M}_{\rm env}}$
$\newcommand{\rmaxenv}{R_{\rm env,max}}$
$\newcommand{\rminenv}{R_{\rm env,min}}$
$\newcommand{\thetac}{\theta_{C}}$
$\newcommand{\mdotdisk}{\dot{M}_{\rm disk}}$
$\newcommand{\rmaxdisk}{R_{\rm disk,max}}$
$\newcommand{\rmindisk}{R_{\rm disk,min}}$
$\newcommand{\mdisk}{M_{\rm disk}}$
$\newcommand{\lbol}{L_{\rm bol}}$
$\newcommand{\tbol}{T_{\rm bol}}$
$\newcommand{\lsl}{L_{\rm smm}/L_{\rm bol}}$
$\newcommand{\lsmm}{L_{\rm smm}}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\mx}{{\rm max}}$
$\newcommand{\planck}{\textit{Planck}\xspace}$
$\newcommand{\vlsr}{V_{\rm LSR}\xspace}$
$\newcommand{\mum}{\micron\xspace}$
$\newcommand{◦ee}{\mbox{^{\circ}}\xspace}$
$\newcommand{\minflow}{\dot{\textup{M}}_{\textup{tot,in}}\xspace}$
$\newcommand{\HII}{H {\sc ii} }$
$\newcommand{\novel}{\multicolumn{1}{c}{\textemdash}}$
$\newcommand{\ntot}{N_{\rm tot}\xspace}$
$\newcommand{\mtot}{M_{\rm tot}\xspace}$
$\newcommand{\msunyr}{\hbox{\hbox{\rm M}_{\odot} \rm{yr}^{-1}}\xspace}$
$\newcommand{\amy}{\textcolor{purple}}$
$\newcommand{\nico}{\textcolor{cyan}}$</div>



<div id="title">

# ALMA-IMF XVIII: The assembly of a star cluster: Dense $\nhp$ (1-0)     kinematics in the massive G351.77 protocluster

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.09843-b31b1b.svg)](https://arxiv.org/abs/2410.09843)<mark>Appeared on: 2024-10-15</mark> -  _Submitted in A&A, 28 pages, 31 figures, 4 interactive figures, 7 tables_

</div>
<div id="authors">

N. A. Sandoval-Garrido, et al. -- incl., <mark>P. Garcia</mark>

</div>
<div id="abstract">

**Abstract:** ALMA-IMF observed 15 massive protoclusters capturing multiple spectral lines and the continuum emission. Here we focus on the massive protocluster G351.77 ( $\sim$ 2500 M $_{\odot}$ , estimated from single-dish continuum observations) located at 2 kpc. We trace the dense gas emission and kinematics with $N_2$ H $^+$ (1-0) at $\sim$ 4 kau resolution. We estimate an $N_2$ H $^+$ relative abundance $\sim (1.66 \pm 0.46) \times 10^{-10}$ . We decompose the $N_2$ H $^+$ emission into up to two velocity components, highlighting the kinematic complexity in the dense gas. By examining the position-velocity (PV) and PPV diagrams on small scales, we observe clear inflow signatures (V-shapes) associated with 1.3 mm cores. The most prominent V-shape has a mass inflow rate of $\sim 9.82 \times 10^{-4}$ M $_{\odot}$ yr $^{-1}$ and a short timescale of $\sim$ 15.63 kyr. We also observe V-shapes without associated cores. This suggests both that cores or centers of accretion exist below the 1.3 mm detection limit, and that the V-shapes may be viable tracers of very early accretion and star formation on $\sim$ 4 kau scales. The large-scale PV diagram shows that the protocluster is separated into 2 principal velocity structures separate by $\sim$ 2 km s $^{-1}$ . Combined with smaller scale DCN and $H_2$ CO emission in the center, we propose a scenario of larger scale slow contraction with rotation in the center based on simple toy models. This scenario is consistent with previous lines of evidence, and leads to the new suggestion of outside-in evolution of the protocluster as it collapses. The gas depletion times implied by the V-shapes are short ( $\sim$ 0.3 Myr), requiring either very fast cluster formation, and/or continuous mass feeding of the protocluster. The latter is possible via the Mother Filament G351.77 is forming out of. The remarkable similarities in the properties of G351.77 and the recently published work in G353.41 indicate that many of the physical conditions inferred via the ALMA-IMF $N_2$ H $^+$ observations may be generic to protoclusters.

</div>

<div id="div_fig1">

<img src="tmp_2410.09843/./Figure/11.png" alt="Fig29" width="100%"/>

**Figure 29. -** Integrated intensity and position-velocity (PV) diagrams of
  the Blue- (blue colorbar) and Red- (red color bar) velocity components
  seen in $\nhp$ (1-0). Top left: Spatial distribution of $\nhp$ (1-0) emission in
  G351.77.  The green, black, and magenta $\times$ markers indicate
  the positions of the 9 out of the 18 (see \S \ref{sec:kinematic-analysis-at-small-scales})
  dense cores  ([Louvet, Sanhueza and Stutz 2024]()) ,
  where each color represents the DCN spectral
  classification: single, complex, and non-detected, respectively  ([Cunningham, Ginsburg and Galván-Madrid 2023]()) .
  The $+$ markers indicate the position of the 16 core candidates, proposed on the basis
   on the $\nhp$ PV features observed at scales of
  $\sim$ 0.1 pc (see \S \ref{sec:kinematic-analysis-at-small-scales}
  and Table \ref{tab:VGcores}). Dashed lines indicate the
  positions of the four cores that do not have measurable velocities. The red arrow
  indicates the direction of the Mother Filament.
  The ellipse in the bottom-left represents the
  beam size of the $\nhp$ data. Top right and bottom left: PV diagrams along
  the two perpendicular directions. The arrows indicate the position of the
  dense cores, listed in Table \ref{tab:core_parameters}, in the PV diagram.
  We observe multiple structures,
  such as V-shapes and Straight structures (see text) along the filaments,
  some associated with cores in both position and
  velocity. The orange arrows represent a velocity gradient of
  10 $\kms$ pc$^{-1}$$\approx$ 0.1 Myr. (*fig:pv_diagram1*)

</div>
<div id="div_fig2">

<img src="tmp_2410.09843/./Figure/7.1.png" alt="Fig7.1" width="33%"/><img src="tmp_2410.09843/./Figure/7.2.png" alt="Fig7.2" width="33%"/><img src="tmp_2410.09843/./Figure/7.3.png" alt="Fig7.3" width="33%"/>

**Figure 7. -** Top: Mean velocity map from the final spectral model
  composed of both the 1- and 2-velocity-components. The red arrow display
  the direction of the Mother Filament. Whereas the blue arrow indicates
  the direction of the large-scale velocity gradient measured from the centroid velocities.
  Middle: Centroid velocity
  map of the Blue-velocity component. Bottom: Centroid velocity map of the
  Red-velocity component. The spectra inside the black contour are fit
  with 2-velocity-components. The ellipse in the bottom-left corner represents the beam
  size of the $\nhp$ data. (*fig:velocity_map*)

</div>
<div id="div_fig3">

<img src="tmp_2410.09843/./Figure/15.png" alt="Fig30" width="100%"/>

**Figure 30. -** Integrated intensity and position-velocity (PV) diagram of
  the F1 (blue colorbar), F2 (red colorbar), F3 (green colorbar), and
  F4 (purple colorbar) large-scale structures of $\nhp$ (1-0) into which the
  protocluster has been divided. Left: Spatial distribution of $\nhp$ (1-0)
  emission. Markers and dashed lines are the same as in Fig. \ref{fig:pv_diagram1}.
  The ellipse in the bottom-left
  represents the beam size of the $\nhp$ data. Right: PV diagram. We observe
  multiple large-scale V-shapes and Straight-shapes along the different
  filamentary structures, with some of them associated with cores in
  position and in velocity. The green arrows represent a velocity
  gradient of 10 $\kms$ pc$^{-1}$, which correspond to a timescale
  of $\sim$ 0.1 Myr. (*fig:pvcolores*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.09843"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\exampleConstant}{0.04}$
$\newcommand{◦ee}{^\circ}$
$\newcommand{\methane}{CH_4}$
$\newcommand{\cotwo}{CO_2}$
$\newcommand{\cotwoSignificance}{2.4 \sigma}$
$\newcommand{\chfourSignificance}{2.0 \sigma}$
$\newcommand{\ocsSignificance}{\textcolor{red}{Not favored}}$
$\newcommand{\ngts}{NGTS}$
$\newcommand{\tess}{TESS}$
$\newcommand{\edit}[1]$</div>



<div id="title">

# Possible Carbon Dioxide Above the Thick Aerosols of GJ 1214 b

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.10183-b31b1b.svg)](https://arxiv.org/abs/2410.10183)<mark>Appeared on: 2024-10-15</mark> -  _22 pages, 11 figures, Accepted in ApJL, Please also see a companion paper Ohno et al. (2024)_

</div>
<div id="authors">

E. Schlawin, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** Sub-Neptune planets with radii $\edit$ 1 ${smaller than}$ Neptune (3.9 R $_\oplus$ ) are the most common type of planet known to exist in The Milky Way, even though they are absent in the Solar System.These planets can potentially have a large diversity of compositions as a result of different mixtures of rocky material, icy material and gas accreted from a protoplanetary disk.However, the bulk density of a sub-Neptune, informed by its mass and radius alone, cannot uniquely constrain its composition; atmospheric spectroscopy is necessary.GJ 1214 b, which hosts an atmosphere that is potentially the most favorable for spectroscopic detection of any sub-Neptune, is instead enshrouded in aerosols (thus showing no spectroscopic features), hiding its composition from view at previously observed wavelengths in its terminator.Here, we present a JWST NIRSpec transmission spectrum from 2.8 to 5.1 $\micron$ that shows signatures of $CO_2$ and $CH_4$ , expected at high metallicity.A model containing both these molecules is preferred by 3.3 and 3.6 $\sigma$ as compared to a featureless spectrum for two different data analysis pipelines, respectively.Given the low signal-to-noise of the features compared to the continuum, however, more observations are needed to confirm the $CO_2$ and $CH_4$ signatures and better constrain other diagnostic features in the near-infrared.Further modeling of the planet's atmosphere, interior structure and origins will provide valuable insights about how sub-Neptunes like GJ 1214 b form and evolve.

</div>

<div id="div_fig1">

<img src="tmp_2410.10183/./wavelength_selection_CO2.png" alt="Fig2.1" width="50%"/><img src="tmp_2410.10183/./if_vs_oof_lightcurve_CO2.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** _ Left:_ Wavelength selections for $CO_2$ and Out-of-Feature co-adding.
    We use the chemically-consistent model (purple curve) and a threshold (dashed black line) to select wavelengths that are dominated by $CO_2$ opacity (brown circles) and wavelengths that are outside of features (pink circles).
    Individual spectra are shown for the NRS1 and NRS2 detectors and per observation (Obs 18 NRS1 (blue), Obs 18 NRS2 (orange), Obs 19 NRS1 (green), and Obs 19 NRS2 (red)).
    _ Right:_ Differential light curve inside and outside of significant $CO_2$ features. Each detector and observation's fluxes are summed for wavelengths inside a feature and divided by the fluxes summed outside this feature.
    The same blue, orange, green and red colors are used on the left and right to code each detector and observation.
    Finally, an average in-feature versus out-of-feature light curve across both observations is shown in purple and brown on the bottom for NRS1 and NRS2, respectively.
    The same wavelengths' best fit models are shown as black curves, which use a limb-darkened \texttt{starry} model.
    Linear trends have been removed from all time series and $\edit$1{vertical offsets were applied} for clarity. (*fig:CO2lc*)

</div>
<div id="div_fig2">

<img src="tmp_2410.10183/./Data_2pipeline.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Transmission Spectrum of GJ 1214 b with Two Data Reductions and Illustrative Models:**
    The top panel shows the results of data reduction by the \texttt{tshirt} and \texttt{Eureka!} pipelines, demonstrating that two independent pipelines yield consistent results within $1\sigma$ errorbars.
    The solid black, solid gray and dash-dotted gray lines show the median spectrum from the chemically-consistent model, chemically-agnostic free model, and a flat line (with stellar inhomogeneities), respectively fit to the HST, JWST-NIRSpec (\texttt{tshirt}) and JWST-MIRI panchromatic spectrum.
    The lower panels show the illustrative transmission spectra to exhibit the transit depth shape of relevant molecules -- we create clear isothermal ($T=500 {\rm K}$)  $N_2$ atmospheres with only one gas at a time with a VMR=$10^{-4}$ and repeat for all molecules. (*fig:differentAnalyses*)

</div>
<div id="div_fig3">

<img src="tmp_2410.10183/./lc_2D_all_nrs2.png" alt="Fig10.1" width="50%"/><img src="tmp_2410.10183/./lc_2D_all_nrs1.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** 2D light curves binned to a resolution of R=100 and 40 time bins for NRS2 (top row) and NRS1 (bottom row), with both transits averaged. The normalized flux in percentage (after linearly de-trending at each wavelength) is shown in panel A. The differential time series, after dividing by the broadband for a given detector is shown in ppm in panel B. The differential light curve model is shown in panel C and the residuals of the light curve subtracted by the model are shown in panel D.
    The start of ingress and end of egress times are marked as vertical dashed lines.
    Increased and decreased transit depths relative to the broadband light curve are visible as dark and bright bands between ingress and egress.
    The most noticeable features are increased transit depths near 4.3 \micron and 2.8 \micron and decreased transit depths near 3.6 \micron and 4.6 \micron. (*fig:2Dlc*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.10183"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

219  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
